In [98]:
from scipy import sparse
from sklearn.model_selection import train_test_split

In [96]:
!pip uninstall scikit-learn

^C


In [49]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as npy
from scipy.stats import linregress
from sklearn.model_selection import GroupShuffleSplit
import dataframe_image as dfi

bg = pd.read_feather("boardgamegeek.feather")
np = pd.read_feather("netflix_prize.feather")
m = pd.read_feather("movielens_25m.feather") 
y = pd.read_feather("yahoo_r2_songs.subsampled.feather")

In [50]:
plt.rcParams['figure.dpi'] = 400

In [51]:
bg = bg[bg['user_id'].notnull()]

In [52]:

unique_items = bg['item_id'].unique()
item_id_map = {old: new for new, old in enumerate(unique_items)}
bg['item_id'] = bg['item_id'].map(item_id_map)
n_items = len(unique_items)

In [ ]:
groups = bg['user_id']
gss = GroupShuffleSplit(n_splits=1,test_size=0.25,random_state=42)

In [81]:
train_idx,test_idx = next(gss.split(bg,groups=groups))

In [82]:
train_data = bg.iloc[train_idx]
test_data = bg.iloc[test_idx]

In [56]:
unique_items = train_data['user_id'].unique()
item_id_map = {old: new for new, old in enumerate(unique_items)}
train_data['user_id'] = train_data['user_id'].map(item_id_map)

C:\Users\18ara\AppData\Local\Temp\ipykernel_9508\2099712719.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['user_id'] = train_data['user_id'].map(item_id_map)


In [57]:
n_training_users = len(unique_items)

In [58]:
n_training_users

308530

In [59]:
training_X = sparse.csr_matrix((train_data["rating"],
                                (train_data["user_id"],
                                train_data["item_id"])),
                                shape=(n_training_users, n_items))

In [60]:
test_data.head()

,item_id,user_id,rating
7,0,htwongah,8.0
14,0,Narstrand,7.8
21,0,m1hune2007,7.5
24,0,lemados,7.0
36,0,3villans,7.0


In [83]:
user_counts = test_data['user_id'].value_counts()
valid_users = user_counts[user_counts >= 2].index
filtered_data = test_data[test_data['user_id'].isin(valid_users)]

unique_items = filtered_data['user_id'].unique()
item_id_map = {old: new for new, old in enumerate(unique_items)}
filtered_data['user_id'] = filtered_data['user_id'].map(item_id_map)

test_seen, test_unseen = train_test_split(
    filtered_data,
    test_size=0.5,
    stratify=filtered_data['user_id'],
    random_state=42
)



C:\Users\18ara\AppData\Local\Temp\ipykernel_9508\2087887513.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['user_id'] = filtered_data['user_id'].map(item_id_map)


In [88]:
n_testing_users = test_seen['user_id'].nunique()

In [89]:
testing_X = sparse.csr_matrix((test_seen["rating"],
                                (test_seen["user_id"],
                                test_seen["item_id"])),
                                shape=(n_testing_users, n_items))